### Investigating ploidy

In [1]:
import h5py 
import numpy as np
import ipyrad as ip

In [3]:
data = ip.load_json("/home/deren/Downloads/anchored/base.json")


  loading Assembly: base
  from saved path: ~/Downloads/anchored/base.json


In [10]:
sample = data.samples['29154_superba']
sample.files.database

[]